## NoteBook Para crear El Data Warehouse

In [1]:
import pandas as pd
import numpy as np
import boto3
import configparser
import psycopg2
import pymysql

In [2]:
import sql_queries_H

In [4]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [5]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'), 
                            aws_secret_access_key = config.get('IAM', 'SECRET_KEY'), region_name='us-east-2')

### Verificacion de instancias disponibles para el usuario

In [6]:
rds_instances_ids = []

aws_response = aws_rds_conn.describe_db_instances()

for item in aws_response['DBInstances']:
    rds_instances_ids.append(item['DBInstanceIdentifier'])
    
print(f"Instancias disponibles: {rds_instances_ids}")

Instancias disponibles: ['ecomercedb', 'hechosdbproject']


### Creamos la base de datos

In [7]:
try:
    response = aws_rds_conn.create_db_instance(
                            DBInstanceIdentifier=config.get('HECHOS', 'DB_INSTANCE_ID'),
                            DBName=config.get('HECHOS', 'DB_NAME'),
                            MasterUsername=config.get('HECHOS', 'DB_USERNAME'),
                            MasterUserPassword=config.get('HECHOS', 'DB_PASSWORD'),
                            Port=int(config.get('HECHOS', 'DB_PORT')),
                            DBInstanceClass='db.t3.micro',
                            Engine='mysql',
                            PubliclyAccessible=True,
                            AllocatedStorage=20,
                            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
                        )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!!!', ex)

La instancia ya existe


### Obtenemos el hostname de la instancia

In [8]:
try: 
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('HECHOS','DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print('Error!!!',ex)

hechosdbproject.ctiwauocyj1r.us-east-2.rds.amazonaws.com


In [10]:
### No funciona esta libreria, no ejecutar, dejar de evidencia

try:
    db_pg_conn = pymysql.connect(
                        database=config.get('HECHOS', 'DB_NAME'),
                        user=config.get('HECHOS', 'DB_USERNAME'),
                        password=config.get('HECHOS', 'DB_PASSWORD'),
                        port=int(config.get('HECHOS', 'DB_PORT')),
                        host=RDS_HOSTNAME)
    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries_H.DDL_QUERY)
    db_pg_conn.commit()
except Exception as ex:
    print('Error!!!!', ex)
finally:
    cursor.close()
    db_pg_conn.close()

Error!!!! (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CREATE TABLE Usuario (\n\n    idusuario int primary key,\n    idrol int,\n    nombre' at line 18")


In [12]:
import mysql.connector

In [15]:
try:
    cnx = mysql.connector.connect(user=config.get('HECHOS', 'DB_USERNAME'),
                                  password=config.get('HECHOS', 'DB_PASSWORD'),
                                  host=RDS_HOSTNAME,
                                  database=config.get('HECHOS', 'DB_NAME'))

    cursor = cnx.cursor()
    cursor.execute(sql_queries_H.DDL_QUERY)
except Exception as ex:
    print('Error!!!!', ex)
finally:
    cursor.close()
    cnx.close()